<a href="https://colab.research.google.com/github/prakashnalubolu/Pokemon-battle-predictions/blob/main/weedle_cave_battle_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pokemon-challenge:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2619%2F4359%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240917%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240917T231306Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0ee1569a05f8d570184b2da29ebbcec4861f0896e4ba746f825028055b4da3ad2d4f51fb10d77d15d9ceaebb91f78e61a0a26912e87c549e5f87d1ec347471abcd35d3d147ae37e054f7226b08bc8da454785f89297fcd95167d8ce0843877a503d2e0ffffc32a111b37976485084f916e903305c75c181fd374f6a3ac064832c89789914917ca07f51f934d213680a02ed24340393e38136f58116656dc735496619a7aebb74c6c00bd9a42ba91a8d104344cb8b588a98d3454ef33976ea0cd1b2b70fe02f3f80e5c581e8be472136cd6caf73790380024edfa03fee06b56f16a0843ba7c9357d9f84343eab1fe93039ab0c2ead0188f4c536b06aef76c6159'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 291852 bytes downloaded
Downloaded and uncompressed: pokemon-challenge
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pokemon-challenge/combats.csv
/kaggle/input/pokemon-challenge/tests.csv
/kaggle/input/pokemon-challenge/pokemon.csv


In [3]:
combat_file_path = '/kaggle/input/pokemon-challenge/combats.csv'
combat_data = pd.read_csv(combat_file_path)

#combat_data.describe()

first_few_rows_combat_data = combat_data.head(5)
print(first_few_rows_combat_data)

   First_pokemon  Second_pokemon  Winner
0            266             298     298
1            702             701     701
2            191             668     668
3            237             683     683
4            151             231     151


In [4]:
pokemon_file_path = '/kaggle/input/pokemon-challenge/pokemon.csv'
pokemon_data = pd.read_csv(pokemon_file_path)

pokemon_data.describe()

first_few_rows_pokemon_data = pokemon_data.tail(1)
print(first_few_rows_pokemon_data)

       #       Name Type 1 Type 2  HP  Attack  Defense  Sp. Atk  Sp. Def  \
799  800  Volcanion   Fire  Water  80     110      120      130       90   

     Speed  Generation  Legendary  
799     70           6       True  


In [5]:
tests_file_path = '/kaggle/input/pokemon-challenge/tests.csv'
test_data = pd.read_csv(tests_file_path)

#tests_data.describe()

first_few_rows_tests_data = test_data.head(1)
print(first_few_rows_tests_data)

   First_pokemon  Second_pokemon
0            129             117


In [6]:
import pandas as pd

print("Combat Data Columns:", combat_data.columns.tolist())
print("Pokemon Data Columns:", pokemon_data.columns.tolist())


Combat Data Columns: ['First_pokemon', 'Second_pokemon', 'Winner']
Pokemon Data Columns: ['#', 'Name', 'Type 1', 'Type 2', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary']


In [7]:
import pandas as pd

# Merge combat data with pokemon data for the first Pokémon
combat_data = combat_data.merge(pokemon_data, left_on='First_pokemon', right_on='#', how='left', suffixes=('', '_first'))

# Merge combat data with pokemon data for the second Pokémon
combat_data = combat_data.merge(pokemon_data, left_on='Second_pokemon', right_on='#', how='left', suffixes=('_first', '_second'))

# Feature engineering: differences between the first and second Pokémon's attributes
for feature in ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']:
    combat_data[f'{feature}_diff'] = combat_data[f'{feature}_first'] - combat_data[f'{feature}_second']

# Drop redundant columns (columns with '#' as they are just the ID columns repeated)
combat_data.drop(columns=['#_first', '#_second'], inplace=True)

# Preview the enriched combat data
print(combat_data.head(10))

print("Combat Data Columns:", combat_data.columns.tolist())

   First_pokemon  Second_pokemon  Winner Name_first Type 1_first Type 2_first  \
0            266             298     298   Larvitar         Rock       Ground   
1            702             701     701   Virizion        Grass     Fighting   
2            191             668     668    Togetic        Fairy       Flying   
3            237             683     683     Slugma         Fire          NaN   
4            151             231     151    Omastar         Rock        Water   
5            657             752     657     Joltik          Bug     Electric   
6            192             134     134       Natu      Psychic       Flying   
7             73             545     545     Machop     Fighting          NaN   
8            220             763     763     Pineco          Bug          NaN   
9            302              31      31    Wingull        Water       Flying   

   HP_first  Attack_first  Defense_first  Sp. Atk_first  ...  Sp. Def_second  \
0        50            64   

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from collections import Counter

# Load the data (assuming combat_data and pokemon_data are already loaded and merged as described)
# Split the data into features and target
X = combat_data[['HP_diff', 'Attack_diff', 'Defense_diff', 'Sp. Atk_diff', 'Sp. Def_diff', 'Speed_diff']]
y = combat_data['Winner']

# Encode the target variable correctly
y = combat_data.apply(lambda row: 1 if row['Winner'] == row['First_pokemon'] else 0, axis=1)

# Check class distribution
print("Class distribution in target variable:", Counter(y))

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train the logistic regression model
model = RandomForestClassifier(n_estimators = 100, random_state = 42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=1)
recall = recall_score(y_test, y_pred, zero_division=1)
f1 = f1_score(y_test, y_pred, zero_division=1)
roc_auc = roc_auc_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'ROC AUC Score: {roc_auc}')


print("Test Data Columns:", test_data.columns.tolist())

Class distribution in target variable: Counter({0: 26399, 1: 23601})
Accuracy: 0.95
Precision: 0.9410910599460692
Recall: 0.954555017883442
F1 Score: 0.9477752245665343
ROC AUC Score: 0.9502144252748637
Test Data Columns: ['First_pokemon', 'Second_pokemon']


In [9]:
# Prepare the test data for prediction
test_data = test_data.merge(pokemon_data, left_on='First_pokemon', right_on='#', how='left', suffixes=('', '_first'))
test_data = test_data.merge(pokemon_data, left_on='Second_pokemon', right_on='#', how='left', suffixes=('_first', '_second'))

# Feature engineering for test data
for feature in ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']:
    test_data[f'{feature}_diff'] = test_data[f'{feature}_first'] - test_data[f'{feature}_second']

# Drop redundant columns
test_data.drop(columns=['#_first', '#_second'], inplace=True)

# Standardize the test data
X_test_final = scaler.transform(test_data[['HP_diff', 'Attack_diff', 'Defense_diff', 'Sp. Atk_diff', 'Sp. Def_diff', 'Speed_diff']])

# Make predictions
test_data['Prediction'] = model.predict(X_test_final)

# Map predictions to Pokémon IDs
test_data['Winning_Pokemon'] = test_data.apply(lambda row: row['First_pokemon'] if row['Prediction'] == 1 else row['Second_pokemon'], axis=1)

# Print the result with Pokémon IDs
print(test_data[['First_pokemon', 'Second_pokemon', 'Winning_Pokemon']])



      First_pokemon  Second_pokemon  Winning_Pokemon
0               129             117              129
1               660             211              211
2               706             115              706
3               195             618              618
4                27             656               27
...             ...             ...              ...
9995            216             498              498
9996            113             404              404
9997            493             104              493
9998            643             259              643
9999            678             531              531

[10000 rows x 3 columns]
